In [16]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Efficient Multi-Party Private Set Intersection

### By Malia Kency and John Owens

First, load imports

In [17]:
import protocol
import helpers
import hashes as h
import bloom_filter as bf

Set constant protocol parameters:

   * $NumPlayers$      = Total number of players $P_i$
   * $PlayerInputSize$ = Size of the player's input sets
   * $N_{BF}$          = Length of the Bloom Filter
   * $k$               = Number of hash functions to use
   * $\kappa           = SecParam$ = Security paremater. 40 in the paper's example
   * $N_{Maxones}$     = Maximum number of ones a player is allowed after cut-and-choose
   * $p$               = percentage of total messages to be used for cut-and-choose
   * $a$               = sampling weight of 1s vs. 0s for every $P_i$

Then calculate the rest:
   * $N_{OT}$ = Total number of Random Oblivious Transfers
   * $m_h^1$ = The number of 1's a player chooses
   * $\gamma = gamma$ = Verifies the correct relationship between $p, \kappa, m_h^1$
   * $\gamma^* = gammaStar$ = Verifies the correct relationship between $p, \kappa, N_{OT}$
   * $\varphi$
   * $\pi$
   * $\epsilon$
   * $\lambda$
   

In [18]:
NumPlayers      = 3 
PlayerInputSize = 5
Nbf             = 10
SecParam        = 40
Nmaxones        = 10
bitLength       = 128
p = 0.25
a = 0.3

# Initialize the protocol with the parameters above
Protocol = protocol.new(NumPlayers, Nmaxones, PlayerInputSize, p, a, SecParam, bitLength, Nbf)

k = 2
Not = 67

Simulating players joining protocol. Total: 3
Player 0 created
Player 1 created
Player 2 created

Creating Bloom Filter simulation. BF length: 10


In [19]:
# Perform Random Oblivious Transfer
Protocol.perform_RandomOT()


Simulating Random OT Stage. Performing 67 transfers in total:
                                                   P0  \
0   (P1, Bit: 0, b'?\xfc,\xa2fWo\x97K\xfc9\xa2\x8b...   
1   (P1, Bit: 0, b'\xfb\xca\xba1\xad\x1d?\xeb\x10\...   
2   (P1, Bit: 0, b'\x92i\x1aZ\xdb\xa9\x17Q\x15\xe3...   
3   (P1, Bit: 0, b'a\x91|\x9c\xbeC\x0e\x15\x8eCD\x...   
4   (P1, Bit: 0, b'\xe1\x0c\xb6\xdf~\xb8\xae%\xbb\...   
..                                                ...   
62  (P1, Bit: 0, b'W\x9e\xb0\x91.A\xd5r\xdf\xc3\x8...   
63  (P1, Bit: 0, b'\xd3!\x1b\xa8\xc8d\x8f,\xad)In\...   
64  (P1, Bit: 0, b'\xbe\x1c@~Iz$\xd9\xbf\xc7\xdd\x...   
65  (P1, Bit: 0, b'\xe7\xed\xa1~\x16\xb8_;I_6\xb5\...   
66  (P1, Bit: 0, b'\xa1E<\xf4\xed\xfd\x92Z\xc1\xcf...   

                                                   P0  \
0   (P1, Bit: 1, b'&5\x0fE\xa2\xa2\t{\x00a\xfa \xe...   
1   (P1, Bit: 1, b"\xdf\x10\xd4\xa8'\xf3\xd7v\x13\...   
2   (P1, Bit: 1, b'\x1a\x9b8\x0byL/\x15\xf2#-\xe7\...   
3   (P1, Bit: 1, b'\xa5\

In [20]:
# Get all players one
Protocol.get_AllPlayersOnes()


Counting each player's "1s":
P0 has 134 ones. a * Not: 20.099999999999998
P1 has 25 ones. a * Not: 20.099999999999998
P2 has 19 ones. a * Not: 20.099999999999998


In [21]:
# Perform cut-and-choose
Protocol.performCutandChoose()

In [22]:
# Create an Injective Function
Protocol.create_InjectiveFunctions()


Creating injective function simulation for every Pi:
Player 1 injective function valid
Player 2 injective function valid


In [23]:
# Create Randomized GBF
Protocol.players[0].create_RandomizedGBF(Protocol.hashes)

# Calculate XOR sums
Protocol.players[0].create_XOR_sums([ Protocol.players[1], Protocol.players[2] ])

# Create the summary values to share
vals0 = Protocol.players[0].create_SummaryValsToShare( Protocol.hashes )

In [24]:
# Repeat for Player 1
Protocol.players[1].create_RandomizedGBF(Protocol.hashes)
Protocol.players[1].create_XOR_sums([ Protocol.players[1], Protocol.players[2] ])
vals1 = Protocol.players[1].create_SummaryValsToShare( Protocol.hashes )


In [25]:
# Find the intersection
for i in range(0, len(vals0)):
    for j in range(0, len(vals1)):
        if vals0[i] == vals1[j]:
            print("Intersection at {}".format(vals0[i]))


Intersection at b'B\xd1\xfa\xe3U\xd7\x92\x1c\x00\x0fS\t\xb4\x10N_\x9c\xaa3\x81\xfe\xf9?v\xa4\xbfN\x1d\x83\xf5\xf2v'
